In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.0


In [60]:
!pip install tensorflow==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

# Aplicación de una RNN para generar texto

Las RNN (Redes Neuronales Recurrentes) están diseñadas para trabajar con datos de secuencia.

Ejemplo de transformación de corpus a datos de entrenamiento para una RNN.

```python
corpus = "Quiero un jugo de naranja fresco"
seq_lenght = 3
```

|        x        |    y    |
|:---------------:|:-------:|
|  Quiero un jugo |    de   |
|    un jugo de   | naranja |
| jugo de naranja |  fresco |

#### Importar Librerias

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import re
import time
import numpy as np
import pandas as pd
import tensorflow as tf # https://www.tensorflow.org/install

Vamos a crear un generador de texto usando los discursos de Uribe

In [3]:
df = pd.read_csv("uribe.csv", on_bad_lines='skip')
df.head()

,url,titulo,discurso,fecha,lugar
0,http://web.presidencia.gov.co/discursos/discur...,turistica_24022010,“Yo quiero felicitarlos d...,(Bogotá),Febrero 24 de 2010
1,http://web.presidencia.gov.co/discursos/discur...,lloreda_20012010,“Habríamos querido tener ...,(Bogotá),Enero 20 de 2009
2,http://web.presidencia.gov.co/discursos/discur...,bananeros_29042010,"Medellín, 29 abr (SP). “N...",(Medellín),29 de abril de 2010
3,http://web.presidencia.gov.co/discursos/discur...,ccg264_31012010,“Muy apreciados compatrio...,(Bucaramanga),Enero 31 de 2010
4,http://web.presidencia.gov.co/discursos/discur...,audiovisual_21012010,“Una ...,(Bogotá),Enero 21 de 2009


In [4]:
df.shape

(250, 5)

Corpus

In [5]:
text = df.discurso.str.cat(sep=" ")

In [6]:
len(text)

5409214

# RNN

Primero necesitamos vectorizar el texto. Es decir, convertir el corpus en representación numérica.

Para esto creamos dos tablas de búsqueda: de caracteres a números (`char2idx`) y de números a caracteres (`idx2char`)

In [7]:
vocab = sorted(set(text))
print(f'{len(vocab):,.0f} caracteres únicos')

104 caracteres únicos


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [9]:
# Traducir nuestro texto a números
text_as_int = np.array([char2idx[c] for c in text])

In [10]:
# Ejemplo
np.array([char2idx[c] for c in "Uribe, paraco, Jupyter está berraco"])

array([42, 65, 56, 49, 52,  5,  0, 63, 48, 65, 48, 50, 62,  5,  0, 31, 68,
       63, 72, 67, 52, 65,  0, 52, 66, 67, 87,  0, 49, 52, 65, 65, 48, 50,
       62])

#### La tarea de predicción
Dado un caracter, o una secuencia de caracteres, ¿cuál es el siguiente caracter más probable?

#### Ejemplo

```python
seq_length = 4

Texto = "Hello"

Input = "Hell"
Output = "ello"
```

**Epoch**: Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento

In [11]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # división entera

In [12]:
# Crear el conjunto de datos de entrenamiento
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [14]:
split_input_target("Estas redes neuronales están muy densas")

('Estas redes neuronales están muy densa',
 'stas redes neuronales están muy densas')

In [15]:
dataset = sequences.map(split_input_target)

In [16]:
for i,(input_example, target_example) in  enumerate(dataset.take(2)):
    print("***>>> Example #",i)
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    print()

***>>> Example # 0
Input data:  '                    \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que t'
Target data: '                   \xa0“Yo quiero felicitarlos de todo corazón por esta nueva  Vitrina Turística que ta'

***>>> Example # 1
Input data:  'nto ayuda a Colombia. Quiero felicitar a Anato  (Asociación Colombiana de Agencias de Viajes y Turis'
Target data: 'to ayuda a Colombia. Quiero felicitar a Anato  (Asociación Colombiana de Agencias de Viajes y Turism'



In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f"Paso {i}")
    print(f"  input: {input_idx} ({repr(idx2char[input_idx])})")
    print(f"  expected output: {target_idx} ({repr(idx2char[target_idx])})")
    print()

Paso 0
  input: 61 ('n')
  expected output: 67 ('t')

Paso 1
  input: 67 ('t')
  expected output: 62 ('o')

Paso 2
  input: 62 ('o')
  expected output: 0 (' ')

Paso 3
  input: 0 (' ')
  expected output: 48 ('a')

Paso 4
  input: 48 ('a')
  expected output: 72 ('y')



#### Crear lotes de entrenamiento
Usaste `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, tienes que mezclar los datos y empaquetarlos en lotes.

**Batch size**: Tamaño del lote. El número de meustras antes de que el modelo sea actualizado.

**Buffer size**: Tamaño del buffer. Evita que se baraje todo el conjunto de datos.

In [18]:
BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 100), dtype=tf.int64, name=None), TensorSpec(shape=(256, 100), dtype=tf.int64, name=None))>

#### Crear el modelo
Use `tf.keras.Sequential` para definir el modelo. Este modelo va a tener tres capas:

- `tf.keras.layers.Embedding`: Capa de entrada
- `tf.keras.layers.GRU`: RNN
- `tf.keras.layers.Dense`: Capa de salida

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 512

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [21]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (256, None, 64)           6656      
                                                                 
 gru (GRU)                   (256, None, 512)          887808    
                                                                 
 gru_1 (GRU)                 (256, None, 512)          1575936   
                                                                 
 dense (Dense)               (256, None, 104)          53352     
                                                                 
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


#### Ensayar el modelo

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 104) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (256, None, 64)           6656      
                                                                 
 gru (GRU)                   (256, None, 512)          887808    
                                                                 
 gru_1 (GRU)                 (256, None, 512)          1575936   
                                                                 
 dense (Dense)               (256, None, 104)          53352     
                                                                 
Total params: 2,523,752
Trainable params: 2,523,752
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
sampled_indices

array([102,  93,  89, 102,  73,  58,  67,  66,  66,   5,  20,  24,  41,
        35,  58,  83,  18,  48,  92,   9,  46,  17,  54,  39,  61,  54,
        64,  88, 101,  41,  76,  88,   0,  22,  78,  36,  24,   1,   0,
        62,  36,  54,  15,  63,  82,  68,   8,  40,  51,  36,  87,  54,
        50,  59,  42,  14,  29,  38,  14,   5,  82,  82,  43,  96,  75,
        78,  47,  83,  87,  35,  70,  25,  10,   6,  46,  13, 101,  12,
        93,  34,  51,  27,  33,  72,  72,  22,  92,  52,  11,  95, 100,
        28,  37,  43,  69,  46,  34,  77,  31,  34])

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'trabajadores, el   capítulo del respeto a las normas ambiéntales, el capítulo del respeto a los   de'

Next Char Predictions: 
 '•óç•zktss,;CTNkÉ9añ0Y8gRngqä”T¡ä A°OC! oOg6pÁu/SdOágclU5HQ5,ÁÁV–\xa0°ZÉáNwD1-Y4”3óMdFLyyAñe2ü“GPVvYM¬JM'


# Basura

Como podemos observar, el modelo por ahora sólo produce basura. Por eso lo tenemos que entrenar.

In [27]:
# Función de perdida para optimizar el modelo
def loss(labels, logits):
      return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [28]:
model.compile(optimizer='adam', loss=loss)

Entrenar una red neuronal es demorado, por eso debemos ir guardando el progreso

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints2/'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

Entrenar el modelo
<center><img src='https://github.com/vivianamarquez/unicomfacauca-ai-2024/blob/main/imagenes/waiting.jpg?raw=true'></center>

In [30]:
# Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento
EPOCHS=10

#model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # Comentar esta linea la primera vez
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
209/209 [==============================] - 1565s 7s/step - loss: 2.6069
Epoch 2/10
209/209 [==============================] - 1570s 8s/step - loss: 1.7125
Epoch 3/10
121/209 [================>.............] - ETA: 10:44 - loss: 1.3951

KeyboardInterrupt: 

Cargar modelo

In [31]:
p_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
p_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
p_model.build(tf.TensorShape([1, None]))

Hacer predicciones

In [32]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return(start_string + ''.join(text_generated))

In [33]:
print(generate_text(p_model, start_string=u"Trabajar, trabajar y trabajar "))

Trabajar, trabajar y trabajar con la Universidad Jombengran demplonectos y por  cián antesicamina, que   para soluce Leto   hustraro afúncas que la hay má heces. El  quodos la sirticional másporía habías compañerés, padara   paísuro del tien importancemos Uribantes. Ahucio nuestria rues:   magin de las abregatado mezoras, compatativo plansa’.                     En merpacios.  brabajidan tisa secar que la va vacer acreciados: a Muya acerecural porido teneralís.                 infrasugratal de riferción, tampal   nos seño Lanido embresetal a manto ha apredica del inda securdo lan las calral buncos empimo está prospetlencia demeción, tipar a Cracisidad dar gemera, llegó   obra que estrurmo pasado que el países el amun riondo la 1299.                     Na   ferialejía ancueles.48 con ehardiente Maremo ha dijec Ambera de la infertay                          Yo de medos emplesancias   adentras 7,3 millones pregunta porsíanon surmamentar de compal.                       Los regosocrá en e